### <font color='blue'> Import the libraries </font>

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from autogluon import TabularPrediction as task

### <font color='blue'> Read the preprocessed data </font>

In [6]:
data=pd.read_csv('/home/trabeya/cse_ml/data/train_prep.csv')

In [7]:
data=data.drop(['meter_waiting_till_pickup'],axis=1)

In [8]:
data['fare']=data['fare']-data['additional_fare']

NumExpr defaulting to 8 threads.


In [9]:
y=data[['label']].values

In [10]:
X=data.drop('label',axis=1).values

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

In [12]:
file_path='../data'

In [13]:
data = task.Dataset(file_path=f'{file_path}/train_prep.csv') 

Loaded data from: ../data/train_prep.csv | Columns = 11 / 11 | Rows = 16968 -> 16968


In [14]:
data.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,label,hour_of the_day_pickpup,hour_of the_day_drop,day_of_the_week_pickup,distance
0,10.5,834.0,56.0,0.0000,64.0,270.32,0,0,0,4,5
1,10.5,791.0,47.0,0.0000,134.0,197.85,0,0,1,4,3
2,10.5,1087.0,80.0,0.0000,61.0,301.64,0,1,1,4,6
3,10.5,598.0,271.0,15.6638,68.0,82.30,0,2,2,4,0
4,10.5,3407.0,182.0,0.0000,112.0,1065.02,0,5,6,4,24


In [15]:
label_column='label'

In [16]:
dir = 'model' # specifies folder where to store trained models
predictor = task.fit(train_data=data, label=label_column, output_directory=dir,num_bagging_folds=10)

Beginning AutoGluon training ...
AutoGluon will save models to model/
Train Data Rows:    16968
Train Data Columns: 11
Preprocessing data ...
Here are the 2 unique label values in your data:  [0, 1]
AutoGluon infers your prediction problem is: binary  (because only two unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Train Data Class Count: 2
Feature Generator processed 16968 data points with 10 features
Original Features:
	float features: 6
	int features: 4
	object features: 0
Generated Features:
	int features: 0
All Features:
	float features: 6
	int features: 4
	object features: 0
	Data preprocessing and feature engineering runtime = 0.27s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
AutoGluon w

In [17]:
test_data = task.Dataset(file_path=f'{file_path}/test_prep.csv')

Loaded data from: ../data/test_prep.csv | Columns = 10 / 10 | Rows = 8576 -> 8576


In [18]:
predict_test = predictor.predict(test_data)


In [19]:
submission=pd.DataFrame(predict_test,columns=['prediction'])

In [20]:
for i in range(len(submission)):
    if submission['prediction'][i]==0:
        submission['prediction'][i]=1
    else:
        submission['prediction'][i]=0

In [21]:
test=pd.read_csv('/home/trabeya/cse_ml/data/test.csv')

In [22]:
submission['tripid']=test.tripid

In [24]:
final_submition=submission[['tripid','prediction']]

In [17]:
final_submition.to_csv('/home/trabeya/cse_ml/data/final_submition_autoGluon_v2.csv',index=False)